<a href="https://colab.research.google.com/github/shahzad-r1zv1/LocalLLM_experiments/blob/main/7s.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q pandas tqdm matplotlib reportlab python-pptx requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 22.1 MB/s eta 0:00:00


In [3]:
# ============================== ONE-CELL 7S + OLLAMA (COLAB, WRC EDITION) ==============================
# Installs Ollama, pulls up to 3 models, runs selected 7S prompts across models (WRC wording),
# consolidates results, and saves JSON/Markdown outputs in /content.

import os, subprocess, shlex, time, requests, json, re, textwrap, datetime
from typing import Dict, List

# ---------------------------
# 0) BASIC CONFIG (EDIT THESE)
# ---------------------------
# Organization inputs used to fill prompt placeholders
ORG = {
    "name": "QA Automation Org",
    "industry": "Software / DevTools",
    "size_employees": 180,
    "annual_revenue": "N/A (cost center)",
    "locations": "Toronto; Remote",
    "current_context": "Transitioning from phased QA to DevSecOps; aiming for predictive quality and CI/CD-first culture.",
    "key_challenges": "Inconsistent test coverage; siloed defect data; unclear decision rights; skills gap in pipeline engineering.",
    "recent_changes": "Trunk-based dev in 2 squads; feature flags/canary pilots; Playwright + contract tests on 2 critical APIs.",
    # Optional fields for specific prompts (leave as "" if unknown)
    "strategy_statement": "Ship faster with confidence via automated quality gates and proactive risk analytics.",
    "strategic_goals": "12 months: 70% automated coverage; 24m: 95% critical path; MTTR<1hr; DORA elite.",
    "kpis": "DORA metrics, escaped defect rate, mean time to detect, e2e reliability SLOs.",
    "market_position": "Internal enablement platform within a large enterprise.",
    "competitive_advantages": "Domain expertise; test data virtualization; unified telemetry.",
    "target_segments": "Product teams building microservices/APIs.",
    "value_proposition": "Fewer incidents, faster releases, higher trust in automation.",
    "org_chart": "VP Eng → Dir QA/Platform → QA Enablement, SDET, Observability pods; squads map to product lines.",
    "layers": "VP → Director → Managers → ICs (4 layers)",
    "span_of_control": "Managers: 6-8 ICs avg",
    "decision_rights": "Product owns scope; QA owns quality standards; Dev owns implementation; shared release gates.",
    "xfunc": "Release Council; Reliability Guild; Architecture Forum.",
    "geo": "Toronto hub + distributed remote",
    "restructuring": "Evolving towards platform team model; decoupling test infra from app squads."
}

# Which prompts to run (choose from keys below)
RUN_PROMPTS = [
    "PROMPT_1_FULL_7S",
    "PROMPT_2_STRATEGY",
    "PROMPT_3_STRUCTURE",
    "PROMPT_4_SYSTEMS",
    "PROMPT_5_SHARED_VALUES",
    "PROMPT_6_SKILLS",
    "PROMPT_7_STYLE",
    "PROMPT_8_STAFF",
    "PROMPT_9_ALIGNMENT",
    "PROMPT_10_CHANGE",
    "PROMPT_11_DIGITAL",
    "PROMPT_12_BENCH",
    "PROMPT_13_GAPS",
    "PROMPT_14_INTEGRATION",
    "PROMPT_15_EXEC_SUMMARY",
    "MEGA_PROMPT"
]

In [4]:
# Models to try pulling (we’ll use what actually succeeds), "qwen3:8b"
# REQUESTED_MODELS = ["qwen3", "mistral", "llama3"]
#REQUESTED_MODELS = ["qwen3", "phi3", "llama3"]
# Common alias fixes (Ollama doesn't have "gemma3")
# MODEL_ALIASES = {
#     "gemma3": "gemma2:9b-instruct",
#     "gemma2-9b": "gemma2:9b-instruct",
#     "gemma2-2b": "gemma2:2b-instruct",
#     "gemma7b": "gemma:7b-instruct",
#     "llama3.1": "llama3.1",
#      # pass-through example
# }

# # --- Pull models asynchronously ---
# models = ["mistral", "llama3", "gemma2:2b-instruct"]  # use gemma2:2b or 9b instead of gemma3
# models = ["qwen3", "phi3", "llama3"]

MODEL_ALIASES = {
    "gemma3": "gemma2:9b-instruct",
    "gemma2": "gemma2:2b-instruct",
    "llama3": "llama3",
    "llama3.1": "llama3.1",
    "mistral": "mistral",
    "phi4":"phi4",
    "qwen3" : "qwen3:14b",
    "gemma7b": "gemma:7b-instruct",
}
REQUESTED_MODELS = ["mistral", "llama3", "phi4"]
REQUESTED_MODELS = [MODEL_ALIASES.get(m.lower(), m) for m in REQUESTED_MODELS]

# Output paths
OUT_DIR = "/content"
BUNDLE_JSON = os.path.join(OUT_DIR, "7S_bundle.json")
REPORT_MD   = os.path.join(OUT_DIR, "7S_report.md")

In [5]:
# ===============================================================
# 🚀 OLLAMA INSTALL / START / FAST MODEL PULLER (Colab Optimized)
# ===============================================================
import subprocess, shlex, requests, time, os, sys, json

# --- CONFIG ---
OLLAMA_DIR = "/content/drive/MyDrive/ollama_data"
os.environ["OLLAMA_HOME"] = OLLAMA_DIR
os.environ["OLLAMA_MODELS"] = os.path.join(OLLAMA_DIR, "models")
os.makedirs(os.environ["OLLAMA_MODELS"], exist_ok=True)

REQUESTED_MODELS = ["gpt-oss", "mistral", "llama3"]
REQUESTED_MODELS = ["mistral", "llama3", "gemma2"]
REQUESTED_MODELS = ["gpt-oss", "mistral", "llama3"]
REQUESTED_MODELS = ["gpt-oss", "mistral", "llama3", "gemma2", "qwen3:14b"]

REQUESTED_MODELS = [
    "gpt-oss",
    "mistral",
    "llama3",
    "gemma2",
    "qwen3:14b",
]

# --- UTILITIES ---
def run(cmd, check=True, quiet=False):
    if isinstance(cmd, str): cmd = shlex.split(cmd)
    p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    if not quiet: print(p.stdout.strip())
    if check and p.returncode != 0:
        raise RuntimeError(f"CMD FAIL: {' '.join(cmd)}\n----\n{p.stdout}")
    return p.stdout.strip()

def ollama_installed():
    try:
        out = run("ollama --version", check=False, quiet=True)
        return "ollama" in out.lower()
    except Exception: return False

def ollama_running():
    try:
        r = requests.get("http://127.0.0.1:11434/api/tags", timeout=3)
        return r.status_code == 200
    except Exception:
        return False

# --- 1️⃣ INSTALL ---
if ollama_installed():
    print("✅ Ollama already installed.")
else:
    print("📦 Installing Ollama (progressive)...")
    subprocess.run(
        "curl -fsSL https://ollama.com/install.sh | sh",
        shell=True, check=False
    )
    print("✅ Installation complete.")

# --- 2️⃣ START SERVER ---
if not ollama_running():
    print("🚀 Starting ollama serve in background...")
    subprocess.Popen(
        ["ollama", "serve"],
        stdout=open("/tmp/ollama.log", "w"),
        stderr=subprocess.STDOUT,
        text=True
    )
    for i in range(60):
        if ollama_running(): break
        print(".", end="", flush=True); time.sleep(2)
    print("\n✅ Ollama API ready at http://127.0.0.1:11434")
else:
    print("✅ Ollama already running.")

# --- 3️⃣ FAST NATIVE PULL ---
def model_exists(model_name: str) -> bool:
    try:
        out = run("ollama list", check=False, quiet=True)
        return model_name.split(":")[0] in out
    except Exception:
        return False

print(f"\n📥 Downloading models to: {os.environ['OLLAMA_MODELS']}\n")
for model in REQUESTED_MODELS:
    if model_exists(model):
        print(f"✅ {model} already exists, skipping.\n")
        continue
    print(f"⬇️ Pulling {model} ...")
    os.system(f"OLLAMA_MODELS='{os.environ['OLLAMA_MODELS']}' ollama pull {shlex.quote(model)}")

# --- 4️⃣ VERIFY ---
print("\n📋 Final list of models:")
os.system("ollama list")

# --- 5️⃣ SUMMARY ---
print("\n🧾 Summary:")
summary = {
    "models_dir": os.environ["OLLAMA_MODELS"],
    "downloaded_models": [m for m in REQUESTED_MODELS if model_exists(m)]
}
print(json.dumps(summary, indent=2))
print("\n✅ All done! Your models are stored in:", summary["models_dir"])


📦 Installing Ollama (progressive)...
✅ Installation complete.
🚀 Starting ollama serve in background...
...
✅ Ollama API ready at http://127.0.0.1:11434

📥 Downloading models to: /content/drive/MyDrive/ollama_data/models

✅ gpt-oss already exists, skipping.

✅ mistral already exists, skipping.

✅ llama3 already exists, skipping.

✅ gemma2 already exists, skipping.

✅ qwen3:14b already exists, skipping.


📋 Final list of models:

🧾 Summary:
{
  "models_dir": "/content/drive/MyDrive/ollama_data/models",
  "downloaded_models": [
    "gpt-oss",
    "mistral",
    "llama3",
    "gemma2",
    "qwen3:14b"
  ]
}

✅ All done! Your models are stored in: /content/drive/MyDrive/ollama_data/models


In [6]:

# Base endpoints
BASE = "http://127.0.0.1:11434"
TAGS = f"{BASE}/api/tags"
CHAT = f"{BASE}/api/chat"
GEN  = f"{BASE}/api/generate"
print(f"🌐 Connected to Ollama at {BASE}")
PRIMARY_MODEL = REQUESTED_MODELS[0]

🌐 Connected to Ollama at http://127.0.0.1:11434


In [7]:
# ---------------------------
# 1️⃣ INSTALL & START OLLAMA (idempotent)
# ---------------------------
import subprocess, shlex, requests, time, os

def run(cmd, check=True, quiet=False):
    """Run a shell command and optionally suppress output."""
    if isinstance(cmd, str):
        cmd = shlex.split(cmd)
    p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    if not quiet:
        print(p.stdout.strip())
    if check and p.returncode != 0:
        raise RuntimeError(f"CMD FAIL: {' '.join(cmd)}\n----\n{p.stdout}")
    return p.stdout.strip()

def ollama_installed():
    try:
        out = run("ollama --version", check=False, quiet=True)
        return "ollama" in out.lower() or "version" in out.lower()
    except Exception:
        return False

def ollama_running():
    try:
        r = requests.get("http://127.0.0.1:11434/api/tags", timeout=3)
        return r.status_code == 200
    except Exception:
        return False

# 1. Install Ollama if needed
if ollama_installed():
    print("✅ Ollama already installed.")
else:
    print("📦 Installing Ollama…")
    import subprocess, shlex, sys, time

    # Use non-silent curl (remove -s) so progress bar appears
    install_cmd = "bash -lc 'curl -fSL https://ollama.com/install.sh | sh'"

    # Stream output in real time so you can see download progress
    proc = subprocess.Popen(install_cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

    last_update = time.time()
    for line in iter(proc.stdout.readline, ''):
        if line.strip():
            # throttle slightly so Colab output remains responsive
            if time.time() - last_update > 0.2:
                sys.stdout.write(line)
                sys.stdout.flush()
                last_update = time.time()

    proc.wait()
    if proc.returncode == 0:
        print("✅ Ollama installation complete.")
    else:
        print(f"❌ Installation exited with code {proc.returncode}. Check above logs.")

    run("bash -lc 'curl -fsSL https://ollama.com/install.sh | sh'", check=False, quiet=True)
    print("✅ Installation complete.")

# 2. Start Ollama service (if not already running)
if ollama_running():
    print("✅ Ollama service already running.")
else:
    print("🚀 Starting ollama serve…")
    run("bash -lc 'pkill -f \"ollama serve\" || true'", check=False, quiet=True)
    log_path = "/tmp/ollama.log"
    logf = open(log_path, "w")
    proc = subprocess.Popen(["ollama", "serve"], stdout=logf, stderr=subprocess.STDOUT, text=True)

    print("⏳ Waiting for Ollama to become ready", end="")
    ready = False
    for _ in range(60):
        if ollama_running():
            ready = True
            break
        print(".", end="", flush=True)
        time.sleep(2)
    print()

    if not ready:
        print("⚠️ Ollama did not start. Showing last 30 log lines:")
        try:
            print("".join(open(log_path, "r", errors="ignore").readlines()[-30:]))
        except Exception:
            pass
        raise SystemExit("❌ Cannot continue without Ollama.")
    else:
        print("✅ Ollama API ready at http://127.0.0.1:11434")

# Base endpoints
BASE = "http://127.0.0.1:11434"
TAGS = f"{BASE}/api/tags"
CHAT = f"{BASE}/api/chat"
GEN  = f"{BASE}/api/generate"
print(f"🌐 Connected to Ollama at {BASE}")


✅ Ollama already installed.
✅ Ollama service already running.
🌐 Connected to Ollama at http://127.0.0.1:11434


In [8]:
# ---------------------------
# 3) PROMPT TEMPLATES (WRC wording)
# ---------------------------
def fill(s: str, kv: Dict[str,str]) -> str:
    # simple replacement of [FIELD] with kv.get(field_lower)
    def rep(m):
        key = m.group(1).strip().lower().replace(" ", "_")
        return str(kv.get(key, f"N/A: {m.group(0)}"))
    return re.sub(r"\[([^\]]+)\]", rep, s)

PROMPTS = {
"PROMPT_1_FULL_7S": """Act as a WRC (World Renowned Consultancy) consultant conducting a comprehensive 7S analysis.
INPUTS NEEDED:
- Company/Organization name: [NAME]
- Industry: [INDUSTRY]
- Company size (employees): [NUMBER]
- Annual revenue: [REVENUE]
- Geographic presence: [LOCATIONS]
- Current business context: [BRIEF DESCRIPTION]
- Key challenges facing the organization: [LIST 3-5]
- Recent major changes or initiatives: [DESCRIBE]
Analyze all seven elements:
1. Strategy (competitive positioning, strategic priorities, value proposition)
2. Structure (org design, reporting lines, decision rights)
3. Systems (processes, IT systems, workflows)
4. Shared Values (culture, core beliefs, mission/vision)
5. Style (leadership approach, management practices)
6. Staff (workforce composition, capabilities, engagement)
7. Skills (organizational competencies, competitive advantages)
OUTPUT FORMAT:
- Executive summary with key findings
- Detailed analysis of each S (current state assessment)
- Interconnections and dependencies between elements
- Alignment score (1-10) for each element pair
- Top 5 misalignments requiring attention
- Recommendations prioritized by impact and feasibility
""",
"PROMPT_2_STRATEGY": """Evaluate the clarity and effectiveness of our organizational strategy using the 7S model.
INPUTS NEEDED:
- Current strategy statement: [PROVIDE]
- Strategic goals (3-5 year): [LIST]
- Key performance indicators: [LIST]
- Market position: [DESCRIBE]
- Competitive advantages: [LIST]
- Target customer segments: [DESCRIBE]
- Value proposition: [STATEMENT]
Assess our strategy by examining:
1. Clarity and communication throughout organization
2. Alignment with market opportunities
3. Differentiation from competitors
4. Resource allocation alignment
5. Measurability and tracking mechanisms
6. Connection to other 6 S elements
OUTPUT:
- Strategy effectiveness score (1-10) with justification
- Strengths and weaknesses analysis
- Gaps between stated and actual strategy
- Impact assessment on other 6 S elements
- 5 specific recommendations to strengthen strategy
- Communication plan to improve strategy understanding
""",
"PROMPT_3_STRUCTURE": """Analyze our organizational structure's alignment with strategic goals.
INPUTS NEEDED:
- Current org chart: [DESCRIBE HIERARCHY]
- Number of management layers: [NUMBER]
- Span of control averages: [NUMBERS]
- Decision-making authority levels: [DESCRIBE]
- Cross-functional teams/committees: [LIST]
- Geographic/divisional structure: [DESCRIBE]
- Recent restructuring efforts: [DESCRIBE IF ANY]
Evaluate:
1. Structure-strategy fit
2. Decision-making speed and effectiveness
3. Communication flow efficiency
4. Collaboration barriers
5. Duplication or gaps in responsibilities
6. Flexibility for future growth
OUTPUT:
- Structure effectiveness rating with evidence
- Organizational design recommendations
- Proposed org chart modifications
- Impact analysis on systems and staff
- Implementation roadmap for structural changes
- Risk assessment of proposed changes
""",
"PROMPT_4_SYSTEMS": """Assess the effectiveness of operational and management systems.
INPUTS NEEDED:
- Core business processes: [LIST]
- IT systems and platforms: [LIST]
- Performance management systems: [DESCRIBE]
- Financial/budgeting processes: [DESCRIBE]
- Quality control systems: [DESCRIBE]
- Communication systems: [DESCRIBE]
- Decision-making processes: [DESCRIBE]
- Knowledge management systems: [DESCRIBE]
Analyze:
1. Process efficiency and effectiveness
2. System integration and data flow
3. Automation opportunities
4. Performance tracking capabilities
5. User satisfaction and adoption
6. Alignment with strategic objectives
OUTPUT:
- Systems maturity assessment (1-5 scale per system)
- Critical system gaps and redundancies
- Process optimization opportunities
- Technology upgrade recommendations
- Implementation priority matrix
- ROI estimates for system improvements
""",
"PROMPT_5_SHARED_VALUES": """Identify and evaluate core shared values driving culture and decision-making.
INPUTS NEEDED:
- Stated mission/vision/values: [PROVIDE]
- Employee survey results: [KEY FINDINGS]
- Leadership behaviors observed: [EXAMPLES]
- Decision-making patterns: [DESCRIBE]
- Customer feedback themes: [SUMMARIZE]
- Internal communication samples: [PROVIDE EXAMPLES]
- Recognition and reward criteria: [LIST]
Examine:
1. Stated vs. lived values gap analysis
2. Values alignment across hierarchy
3. Values impact on behaviors
4. Cultural strengths and toxicities
5. Values-strategy alignment
6. Employee value perception
OUTPUT:
- Core values identification (top 5 actual vs. stated)
- Cultural health score with supporting evidence
- Values-behavior alignment matrix
- Cultural transformation requirements
- Values reinforcement action plan
- Measurement framework for cultural change
""",
"PROMPT_6_SKILLS": """Evaluate critical skills and competencies across the organization.
INPUTS NEEDED:
- Current workforce skills inventory: [CATEGORIES]
- Strategic capability requirements: [LIST]
- Competitor capabilities: [BENCHMARK DATA]
- Training and development programs: [DESCRIBE]
- Performance review data: [KEY METRICS]
- Skills gaps identified by managers: [LIST]
- Future skill requirements (3-5 years): [ANTICIPATE]
Assess:
1. Current vs. required skills gaps
2. Core competency strengths
3. Competitive skill advantages/disadvantages
4. Skills development effectiveness
5. Knowledge transfer mechanisms
6. Innovation and adaptation capabilities
OUTPUT:
- Skills heat map (current vs. required)
- Critical skills gap analysis with risk levels
- Competency development roadmap
- Make/buy/partner talent decisions
- L&D investment recommendations
- Skills KPIs and tracking mechanisms
""",
"PROMPT_7_STYLE": """Analyze the dominant leadership style and its impact on performance.
INPUTS NEEDED:
- Leadership team composition: [DESCRIBE]
- Leadership assessment data: [IF AVAILABLE]
- Employee engagement scores: [PROVIDE]
- Decision-making examples: [3-5 CASES]
- Communication patterns: [DESCRIBE]
- Change management approaches: [EXAMPLES]
- Succession planning status: [DESCRIBE]
Evaluate:
1. Predominant leadership styles
2. Leadership effectiveness metrics
3. Style-strategy alignment
4. Leadership impact on culture
5. Decision-making patterns
6. Leadership development needs
OUTPUT:
- Leadership style profile with strengths/weaknesses
- Leadership effectiveness score (1-10)
- Style-situation fit analysis
- Leadership development priorities
- Succession planning recommendations
- Leadership behavior change roadmap
""",
"PROMPT_8_STAFF": """Review workforce composition, recruitment, and retention strategies.
INPUTS NEEDED:
- Total headcount and demographics: [PROVIDE]
- Organizational structure by function: [BREAKDOWN]
- Turnover rates by level/function: [DATA]
- Time-to-fill metrics: [AVERAGES]
- Employee engagement scores: [PROVIDE]
- Compensation benchmarking: [POSITION VS MARKET]
- Talent pipeline status: [DESCRIBE]
- Diversity metrics: [PROVIDE]
Analyze:
1. Workforce composition vs. strategic needs
2. Talent acquisition effectiveness
3. Retention risks and drivers
4. Engagement and productivity levels
5. Diversity, equity, and inclusion status
6. Workforce planning adequacy
OUTPUT:
- Workforce health scorecard
- Critical talent risks and mitigation plans
- Recruitment strategy optimization
- Retention program enhancements
- Workforce planning recommendations
- HR metrics dashboard design
""",
"PROMPT_9_ALIGNMENT": """Evaluate how well all seven elements of the 7S Framework align.
INPUTS NEEDED:
- Brief assessment of each S element: [PROVIDE STATUS]
- Recent organizational changes: [LIST]
- Performance metrics trends: [LAST 2 YEARS]
- Strategic priorities: [TOP 5]
- Known pain points: [DESCRIBE]
- Success stories: [EXAMPLES]
Assess:
1. Element interdependencies and conflicts
2. Alignment scoring for each element pair (21 combinations)
3. Reinforcing vs. conflicting relationships
4. Impact of misalignments on performance
5. Root cause analysis of gaps
6. Synergy opportunities
OUTPUT:
- 7S alignment matrix with scores
- Critical misalignment identification
- Dependency map visualization
- Prioritized realignment initiatives
- Change sequencing recommendations
- Alignment monitoring framework
""",
"PROMPT_10_CHANGE": """Use the 7S Framework to analyze organizational readiness for change.
INPUTS NEEDED:
- Planned change initiative: [DESCRIBE]
- Change timeline and scope: [PROVIDE]
- Previous change efforts: [SUCCESS/FAILURE EXAMPLES]
- Stakeholder groups affected: [LIST]
- Current change capability maturity: [ASSESS 1-5]
- Resource availability: [BUDGET/PEOPLE]
- Risk tolerance: [LOW/MEDIUM/HIGH]
Analyze each S element for:
1. Current state change readiness
2. Required changes per element
3. Resistance points and drivers
4. Change capability gaps
5. Success enablers and barriers
6. Change impact assessment
OUTPUT:
- Change readiness score by S element
- Resistance heat map
- Change impact assessment matrix
- Stakeholder engagement strategy
- Change roadmap with quick wins
- Risk mitigation plan
- Success metrics framework
""",
"PROMPT_11_DIGITAL": """Apply the 7S Framework to assess digital transformation readiness and impact.
INPUTS NEEDED:
- Current digital maturity: [ASSESS 1-5]
- Digital strategy/initiatives: [DESCRIBE]
- Technology infrastructure: [CURRENT STATE]
- Digital skills inventory: [ASSESS]
- Data and analytics capabilities: [DESCRIBE]
- Customer digital expectations: [SUMMARIZE]
- Competitor digital positioning: [BENCHMARK]
Evaluate digital impact on:
1. Strategy (digital business models)
2. Structure (agile organization needs)
3. Systems (technology architecture)
4. Shared Values (digital culture)
5. Style (digital leadership)
6. Staff (digital talent)
7. Skills (digital capabilities)
OUTPUT:
- Digital maturity assessment by S element
- Digital transformation gaps and priorities
- Technology investment recommendations
- Digital culture transformation plan
- Reskilling/upskilling requirements
- Digital KPIs and governance model
- Transformation roadmap with milestones
""",
"PROMPT_12_BENCH": """Compare our 7S profile against key competitors.
INPUTS NEEDED:
- Top 3-5 competitors: [LIST]
- Competitive intelligence available: [SUMMARIZE]
- Industry best practices: [DESCRIBE]
- Our performance vs. competitors: [METRICS]
- Competitive advantages/disadvantages: [LIST]
- Market position: [DESCRIBE]
Benchmark:
1. Strategy effectiveness comparison
2. Organizational agility assessment
3. Operational excellence comparison
4. Cultural differentiation analysis
5. Leadership capability comparison
6. Talent competitiveness evaluation
7. Innovation capability assessment
OUTPUT:
- Competitive 7S comparison matrix
- Competitive advantage/disadvantage analysis
- Best practice identification
- Competitive gaps requiring closure
- Differentiation opportunities
- Competitive response strategies
- Monitoring and intelligence framework
""",
"PROMPT_13_GAPS": """Identify gaps between current and desired future state across the 7S elements.
INPUTS NEEDED:
- Vision for future state (3-5 years): [DESCRIBE]
- Current state assessment: [SUMMARIZE BY S]
- Strategic objectives: [LIST]
- Performance targets: [SPECIFY]
- Market/industry trends: [IDENTIFY]
- Stakeholder expectations: [DESCRIBE]
Analyze:
1. Current state baseline for each S
2. Future state requirements per S
3. Gap magnitude and complexity
4. Interdependency impact analysis
5. Resource requirements for gap closure
6. Timeline and sequencing needs
OUTPUT:
- Current vs. future state comparison table
- Gap severity assessment (critical/high/medium/low)
- Gap closure difficulty matrix
- Investment requirements estimate
- Transformation roadmap with phases
- Quick wins vs. long-term initiatives
- Success metrics and milestones
""",
"PROMPT_14_INTEGRATION": """Combine 7S insights with SWOT or PESTLE analysis.
INPUTS NEEDED:
- 7S assessment summary: [PROVIDE]
- SWOT analysis: [IF AVAILABLE]
- PESTLE factors: [IF AVAILABLE]
- Strategic options under consideration: [LIST]
- Risk factors identified: [LIST]
- Opportunity areas: [DESCRIBE]
Integrate analyses to:
1. Map external factors to internal capabilities
2. Identify strategic option feasibility
3. Assess implementation capabilities
4. Determine competitive positioning
5. Evaluate risk mitigation capacity
6. Prioritize strategic initiatives
OUTPUT:
- Integrated strategy framework
- Strategic option evaluation matrix
- Capability-opportunity alignment map
- Risk-readiness assessment
- Strategic initiative prioritization
- Implementation feasibility scores
- Integrated dashboard design
""",
"PROMPT_15_EXEC_SUMMARY": """Create an executive-level summary of 7S analysis with actionable recommendations.
INPUTS NEEDED:
- Full 7S analysis results: [PROVIDE KEY FINDINGS]
- Strategic context and urgency: [DESCRIBE]
- Available resources: [BUDGET/CAPACITY]
- Board/Executive priorities: [LIST]
- Key stakeholder concerns: [IDENTIFY]
- Success criteria: [DEFINE]
Synthesize:
1. Critical insights from 7S analysis
2. Top 3-5 strategic imperatives
3. Quick wins vs. transformational changes
4. Investment requirements and ROI
5. Risk assessment and mitigation
6. Implementation timeline
OUTPUT:
- 2-page executive summary
- Visual 7S alignment dashboard
- Top 10 recommendations ranked by impact
- Investment and resource requirements
- 90-day, 6-month, and 1-year action plans
- Success metrics and governance model
- Key risks and mitigation strategies
- Next steps and decision requirements
""",
"MEGA_PROMPT": """Act as a senior WRC consultant conducting a comprehensive 7S analysis. Provide a complete organizational assessment with actionable insights.
[Use all context provided above and infer reasonable assumptions where N/A.]
DELIVERABLES:
- Executive Summary; Detailed Analysis by S; Integrated Findings; Strategic Recs; Implementation Plan; Appendices.
"""
}

PROMPTS = {

"PROMPT_1_FULL_7S": """
**Role Definition**
You are a **Senior Management Consultant** from a world-renowned firm (McKinsey/Bain/BCG-style) conducting a comprehensive **7S Organizational Analysis**. Your role is to produce a formal, data-driven, and conversational report assessing alignment across all seven McKinsey 7S elements: Strategy, Structure, Systems, Shared Values, Style, Staff, and Skills.

**Contextual Information**
You will receive contextual data about an organization including:
- Company name, industry, size, and revenue
- Geographic footprint
- Current business environment and key challenges
- Recent changes or strategic initiatives
Use this information to craft a coherent, evidence-based 7S assessment highlighting strengths, misalignments, and actionable improvements.

**Task Description and Goals**
Your goal is to:
1. Conduct a structured analysis of each S.
2. Identify cross-dependencies and systemic misalignments.
3. Evaluate strategic coherence and cultural alignment.
4. Recommend prioritized interventions for improvement.
You are expected to think like a consultant preparing a full internal report for executives.

**Instructional Guidance and Constraints**
- Think step-by-step. Identify insights before forming conclusions.
- Use facts provided; infer only when logically defensible.
- Maintain balanced tone: analytical yet accessible.
- Avoid speculative financial or personnel details.
- Organize findings under consistent, clear section headings.
- Keep paragraphs concise; use bullet points where appropriate.

**Expected Output Format and Examples (if applicable)**
Output must include these sections:
1. **Executive Summary** – 1–2 pages summarizing alignment level, key findings, and top 5 recommendations.
2. **Detailed 7S Assessment** – For each S, include:
   - Definition & relevance
   - Current state
   - Strengths & weaknesses
   - Alignment rating (1–10)
   - Recommended improvements
3. **Interdependencies Analysis** – Identify reinforcing/conflicting relationships among elements.
4. **Top Misalignments & Root Causes** – List top 5 issues with rationale.
5. **Recommendations Roadmap** – Prioritize by impact and feasibility (High/Medium/Low).
6. **Success Metrics** – Define KPIs to monitor post-implementation success.

**Any Additional Notes on Scope or Limitations**
- Do not fabricate numerical data.
- Maintain realistic scope; avoid over-engineering.
- Keep the tone factual, professional, and suitable for senior executives.
""",

"PROMPT_2_STRATEGY": """
**Role Definition**
You are a **Strategic Consultant** specializing in corporate transformation. Your task is to evaluate an organization’s **Strategy** using the McKinsey 7S lens, identifying clarity, coherence, and execution readiness.

**Contextual Information**
Inputs provided will include:
- Current strategy statement and strategic goals (3–5 years)
- KPIs, market position, and competitive advantages
- Target customer segments and value proposition
Use this data to assess the realism, differentiation, and communication of the organization’s strategic direction.

**Task Description and Goals**
Objectives:
1. Assess the clarity, relevance, and differentiation of the strategy.
2. Evaluate its internal communication and execution alignment.
3. Determine its consistency with external opportunities and internal capabilities.
4. Recommend refinements to strengthen competitive positioning.

**Instructional Guidance and Constraints**
- Apply frameworks such as **Porter’s Five Forces**, **Value Proposition Canvas**, and **Balanced Scorecard** where relevant.
- Identify measurable inconsistencies or ambiguities.
- Avoid high-level generalizations; provide concrete, actionable advice.
- Maintain a consultant’s tone—direct, precise, evidence-based.
- Summarize insight before recommending solutions.

**Expected Output Format and Examples (if applicable)**
1. **Executive Summary** – Key takeaways on strategy effectiveness and alignment.
2. **Strategic Clarity Analysis** – How well strategy is articulated and understood.
3. **Market Fit and Differentiation Review** – Comparison vs. competitors.
4. **Execution Alignment Score (1–10)** – Justify scoring based on resource allocation and communication.
5. **Strategic Gaps and Risks** – Highlight mismatches between aspiration and capability.
6. **Recommendations and Communication Plan** – 5 specific, prioritized actions to clarify and cascade strategy.

**Any Additional Notes on Scope or Limitations**
- Focus on strategic clarity and alignment—not deep financial modeling.
- Assume qualitative data suffices; avoid external speculation.
- Tone: confident, advisory, pragmatic.
""",

"PROMPT_3_STRUCTURE": """
**Role Definition**
You are an **Organizational Design Expert** analyzing structural alignment with strategy and operational efficiency. Your report should reflect best practices from McKinsey’s Org Design Framework and Galbraith’s Star Model.

**Contextual Information**
You will be provided:
- Current organization chart
- Management layers, spans of control, and authority levels
- Cross-functional collaboration patterns
- Division, region, or business-unit design
- Any restructuring efforts
Your analysis must uncover inefficiencies, redundancies, and opportunities for agility.

**Task Description and Goals**
1. Evaluate structure–strategy alignment.
2. Identify communication bottlenecks and coordination gaps.
3. Assess flexibility for scaling and innovation.
4. Recommend design adjustments improving clarity and accountability.

**Instructional Guidance and Constraints**
- Think hierarchically and cross-functionally.
- Apply organization-design principles (e.g., decision rights matrix, accountability mapping).
- Quantify findings where possible (e.g., spans of control ratios).
- Include before-and-after implications of recommendations.
- Keep tone analytical yet readable.

**Expected Output Format and Examples (if applicable)**
1. **Executive Summary** – Overview of structure alignment level.
2. **Current State Overview** – Describe hierarchy and reporting logic.
3. **Structural Assessment** – Effectiveness, speed of decision-making, cross-team flow.
4. **Alignment Rating (1–10)** – Based on agility, clarity, accountability.
5. **Proposed Modifications and Impact Analysis** – Org chart or structural concept diagrams (optional).
6. **Implementation Roadmap & Risks** – Stepwise change plan and mitigation.

**Any Additional Notes on Scope or Limitations**
- Do not propose headcount numbers unless provided.
- Avoid cultural or personality judgments.
- Maintain focus on structural logic and efficiency.
""",

"PROMPT_4_SYSTEMS": """
**Role Definition**
You are an **Operations and Systems Consultant** assessing organizational processes, IT systems, and performance-management frameworks through the 7S lens.

**Contextual Information**
Inputs may include:
- Core business processes and system map
- IT platforms, data flows, and integration points
- Budgeting, performance, and quality systems
- Communication and decision-support processes
Your objective is to evaluate systemic alignment with strategic and operational needs.

**Task Description and Goals**
1. Assess system efficiency, integration, and automation potential.
2. Identify process bottlenecks and redundancies.
3. Evaluate governance and data consistency.
4. Recommend improvements that enhance performance, transparency, and scalability.

**Instructional Guidance and Constraints**
- Apply frameworks like **ITIL**, **Lean Six Sigma**, or **Process Maturity Models**.
- Think end-to-end—input → process → output → feedback.
- Quantify maturity (1–5 scale per system).
- Propose automation or digitalization opportunities.
- Avoid over-technical jargon; keep the tone accessible to executives.

**Expected Output Format and Examples (if applicable)**
1. **Executive Summary** – Key findings and priority gaps.
2. **Systems Inventory and Maturity Matrix** – Table of processes vs maturity.
3. **Critical Process Analysis** – 3–5 most impactful processes reviewed.
4. **Integration & Data Flow Assessment** – Identify information silos.
5. **Technology Upgrade and Process Optimization Recommendations.**
6. **ROI Estimates and Implementation Priorities.**

**Any Additional Notes on Scope or Limitations**
- Focus on process logic, not vendor-specific software.
- Assume qualitative system info suffices.
- Avoid speculative financial modeling beyond ROI estimates.
""",

"PROMPT_5_SHARED_VALUES": """
**Role Definition**
You are a **Cultural and Organizational Behavior Consultant** assessing an organization’s **Shared Values**—its core beliefs, purpose, and cultural alignment—with its strategy and performance.

**Contextual Information**
Provided inputs may include:
- Mission, vision, and values statements
- Employee survey findings and engagement data
- Leadership behaviors and decision patterns
- Customer feedback themes and recognition criteria
Use this to diagnose cultural health and alignment between stated and lived values.

**Task Description and Goals**
1. Identify gaps between declared and practiced values.
2. Evaluate how values influence decision-making and employee behavior.
3. Assess cultural strengths and toxicities.
4. Recommend cultural transformation initiatives.

**Instructional Guidance and Constraints**
- Use **Edgar Schein’s Cultural Model** (artifacts, espoused values, underlying assumptions).
- Highlight symbolic and behavioral evidence.
- Be empathetic yet objective.
- Avoid judgmental language; focus on patterns and impact.
- Propose specific reinforcement mechanisms (rewards, communication, leadership rituals).

**Expected Output Format and Examples (if applicable)**
1. **Executive Summary** – Cultural alignment overview.
2. **Values Gap Analysis** – Stated vs Lived Values Matrix.
3. **Cultural Health Score (1–10)** – Justify rating.
4. **Behavioral Evidence & Impact Examples** – Real cases illustrating values in action.
5. **Cultural Transformation Roadmap** – Initiatives to reinforce desired behaviors.
6. **Measurement Framework** – How to track cultural change progress.

**Any Additional Notes on Scope or Limitations**
- Avoid sensitive personal attributions.
- Tone must be constructive and developmental.
- Keep recommendations realistic and implementable within 12–24 months.
"""
}

PROMPTS.update({

"PROMPT_6_SKILLS": """
**Role Definition**
You are a **Talent and Capability Consultant** evaluating the organization’s workforce skills and competencies through the 7S lens. Your role is to provide a data-driven, forward-looking assessment that identifies capability strengths, skill gaps, and development priorities.

**Contextual Information**
Inputs include:
- Workforce skill inventory and categories
- Strategic capability requirements
- Competitor benchmarks
- L&D programs and performance review metrics
- Future skill forecasts (3–5 years)
You will synthesize these into a comprehensive skills landscape analysis.

**Task Description and Goals**
1. Compare current vs. required capabilities across functions.
2. Assess skill depth, adaptability, and innovation capacity.
3. Identify mission-critical skill gaps and associated risks.
4. Recommend strategies for capability building, acquisition, or partnership.

**Instructional Guidance and Constraints**
- Apply frameworks such as **Core Competency Theory** and **Learning Organization Model**.
- Classify gaps by criticality and ease of closure.
- Quantify readiness levels where possible (Low/Med/High).
- Avoid over-detailing individual roles; focus on systemic patterns.
- Maintain advisory tone—precise, objective, supportive.

**Expected Output Format and Examples (if applicable)**
1. **Executive Summary** – Workforce capability snapshot and top gaps.
2. **Skills Heat Map** – Table comparing current vs. required skills.
3. **Critical Gap Analysis** – Risk and business impact ratings.
4. **Competency Development Roadmap** – Phased approach (Immediate/Short/Long term).
5. **Make-Buy-Partner Recommendations** – Talent sourcing strategy.
6. **Learning KPIs Dashboard** – Metrics for ongoing monitoring.

**Any Additional Notes on Scope or Limitations**
- Focus on organizational, not individual, skill data.
- Use realistic development timelines.
- Avoid speculative cost estimates unless provided.
""",

"PROMPT_7_STYLE": """
**Role Definition**
You are a **Leadership Style and Effectiveness Consultant** analyzing how executive and managerial behaviors influence culture, decision-making, and performance.

**Contextual Information**
Inputs may include:
- Leadership team composition
- 360° feedback or engagement scores
- Decision-making examples and communication norms
- Change-management case studies
- Succession planning maturity
Use these to map prevalent leadership styles and their organizational effects.

**Task Description and Goals**
1. Identify dominant leadership styles and their alignment with strategy.
2. Evaluate leadership effectiveness and adaptability.
3. Assess how leadership behavior shapes culture and motivation.
4. Recommend development and succession interventions.

**Instructional Guidance and Constraints**
- Apply models such as **Goleman’s Leadership Styles** and **Transformational Leadership Theory**.
- Use behavioral evidence, not personality speculation.
- Rate leadership effectiveness (1–10) with justification.
- Maintain balanced tone: analytical + constructive.

**Expected Output Format and Examples (if applicable)**
1. **Executive Summary** – Leadership culture and impact overview.
2. **Leadership Style Profile** – Dominant and secondary styles identified.
3. **Effectiveness Assessment** – Strengths & development areas.
4. **Style-Situation Fit Analysis** – How leadership supports or hinders strategy.
5. **Development Priorities and Succession Recommendations.**
6. **Behavior Change Roadmap** – Quick wins & long-term levers.

**Any Additional Notes on Scope or Limitations**
- Avoid personal critique; focus on organizational patterns.
- Assume anonymity and professional discretion.
- Use concise, professional, people-focused language.
""",

"PROMPT_8_STAFF": """
**Role Definition**
You are a **Workforce Planning and HR Strategy Consultant** conducting a 7S assessment of the organization’s Staff element—composition, recruitment, retention, and engagement.

**Contextual Information**
Inputs may include:
- Total headcount and demographic data
- Functional structure and turnover metrics
- Time-to-fill and engagement statistics
- Compensation benchmarking
- Talent pipeline and diversity metrics
Your role is to translate these into actionable workforce insights.

**Task Description and Goals**
1. Evaluate workforce composition vs. strategic needs.
2. Analyze recruitment, retention, and engagement efficiency.
3. Identify talent risks, DEI gaps, and succession vulnerabilities.
4. Recommend optimized workforce strategies.

**Instructional Guidance and Constraints**
- Use frameworks: **Workforce Planning Cycle** and **Employee Value Proposition**.
- Focus on patterns & ratios; avoid naming individuals.
- Quantify attrition, engagement, and pipeline health.
- Keep language executive-readable.

**Expected Output Format and Examples (if applicable)**
1. **Executive Summary** – Workforce health overview.
2. **Workforce Health Scorecard** – Turnover, engagement, DEI, talent readiness.
3. **Critical Talent Risks & Drivers.**
4. **Recruitment and Retention Optimization Plan.**
5. **Future Workforce Planning Recommendations.**
6. **HR Metrics Dashboard Blueprint.**

**Any Additional Notes on Scope or Limitations**
- No individual performance discussion.
- Keep forecasts directional, not predictive.
- Maintain confidentiality assumptions.
""",

"PROMPT_9_ALIGNMENT": """
**Role Definition**
You are a **Senior Strategy Integration Consultant** analyzing the alignment of all seven McKinsey 7S elements to determine systemic coherence and performance impact.

**Contextual Information**
You’ll receive:
- Status summaries for each S
- Performance trends (2 years)
- Recent organizational changes
- Strategic priorities and known pain points
Use these to map reinforcing vs. conflicting relationships across the system.

**Task Description and Goals**
1. Identify interdependencies among the 7 elements.
2. Score alignment between each pair (21 combinations).
3. Highlight critical misalignments and their root causes.
4. Recommend prioritized realignment initiatives.

**Instructional Guidance and Constraints**
- Apply **Systems Thinking** and **Causal Loop Mapping** techniques.
- Think relationally—how one element affects another.
- Quantify alignment (1–10) and interpret implications.
- Keep tone precise, board-ready, and actionable.

**Expected Output Format and Examples (if applicable)**
1. **Executive Summary** – Overall Alignment Index and key themes.
2. **7S Alignment Matrix** – Table of pairwise scores (1–10).
3. **Critical Misalignments & Root Causes.**
4. **Dependency Map Visualization (Narrative).**
5. **Prioritized Realignment Initiatives (Impact × Feasibility Matrix).**
6. **Monitoring and Governance Framework.**

**Any Additional Notes on Scope or Limitations**
- Avoid arbitrary scores; justify with evidence.
- Maintain neutrality; highlight interdependencies objectively.
- Tone: confident and data-driven.
""",

"PROMPT_10_CHANGE": """
**Role Definition**
You are a **Change Management and Transformation Consultant** using the 7S framework to assess organizational readiness for an upcoming change initiative.

**Contextual Information**
Inputs include:
- Description of planned change initiative (scope, timeline)
- Past change successes or failures
- Stakeholder groups and capability maturity (1–5)
- Resource availability and risk tolerance
You will analyze each S element to gauge readiness, barriers, and enablers.

**Task Description and Goals**
1. Evaluate readiness per S element.
2. Identify resistance points and enablers.
3. Assess resource adequacy and leadership sponsorship.
4. Recommend a change roadmap with quick wins and risk mitigation.

**Instructional Guidance and Constraints**
- Apply **ADKAR**, **Kotter’s 8 Steps**, or **Prosci Change Model** as reference.
- Distinguish between technical and cultural readiness.
- Use heat maps and impact matrices for clarity.
- Avoid vague motivational language; be specific and measurable.

**Expected Output Format and Examples (if applicable)**
1. **Executive Summary** – Overall readiness overview.
2. **Readiness by S Element** – Table or narrative scoring (1–5).
3. **Resistance and Risk Heat Map.**
4. **Change Impact Assessment Matrix.**
5. **Stakeholder Engagement Strategy.**
6. **Change Roadmap with Quick Wins and Milestones.**
7. **Success Metrics Framework (Leading & Lagging indicators).**

**Any Additional Notes on Scope or Limitations**
- Do not over-speculate on emotions or psychological factors beyond provided data.
- Keep recommendations realistic within resource constraints.
- Tone: structured, encouraging, decisive.
"""
})

PROMPTS.update({

"PROMPT_11_DIGITAL": """
**Role Definition**
You are a **Digital Transformation Consultant** specializing in enterprise modernization and data-driven strategy. Your mission is to assess an organization’s **digital maturity** and readiness through the McKinsey 7S Framework, identifying gaps, priorities, and enablers of successful digital evolution.

**Contextual Information**
Inputs include:
- Current digital maturity score (1–5)
- Ongoing digital initiatives and technology stack
- Data and analytics capabilities
- Digital skill inventory
- Customer digital expectations and competitor benchmarks
Your task is to integrate these into a cohesive view of the organization’s digital posture.

**Task Description and Goals**
1. Assess digital readiness for each 7S element.
2. Identify cultural, structural, and capability barriers.
3. Evaluate ROI potential and alignment with strategic goals.
4. Recommend a phased transformation roadmap with measurable outcomes.

**Instructional Guidance and Constraints**
- Apply frameworks: **Digital Maturity Model**, **Technology Adoption Curve**, **BCG Digital Acceleration Index**.
- Think holistically—technology, people, processes, culture.
- Quantify maturity per S (1–5) and summarize overall readiness.
- Keep tone executive-level yet forward-looking.

**Expected Output Format and Examples (if applicable)**
1. **Executive Summary** – Current maturity and transformation potential.
2. **Digital Readiness by 7S Element** – Ratings with justifications.
3. **Transformation Gaps and Opportunities.**
4. **Technology and Talent Investment Priorities.**
5. **Digital Culture Change Plan.**
6. **Reskilling Roadmap and KPIs.**
7. **Governance and KPI Framework.**

**Any Additional Notes on Scope or Limitations**
- Avoid suggesting specific vendors or technologies.
- Keep recommendations platform-agnostic.
- Assume qualitative data where numeric inputs are missing.
""",

"PROMPT_12_BENCH": """
**Role Definition**
You are a **Competitive Intelligence Consultant** performing a benchmark analysis of the organization’s 7S profile against top competitors to identify relative strengths, weaknesses, and opportunities.

**Contextual Information**
You’ll receive:
- List of 3–5 key competitors
- Available competitive intelligence and market data
- Performance metrics and market positions
- Competitive advantages and disadvantages
You will synthesize these to reveal differentiation opportunities and best practices.

**Task Description and Goals**
1. Compare all 7S elements across competitors.
2. Identify where the organization leads or lags.
3. Highlight competitive advantages to sustain and gaps to close.
4. Recommend actionable competitive responses.

**Instructional Guidance and Constraints**
- Use structured frameworks: **Benchmarking Wheel**, **Competitor Capability Matrix**, **Porter’s Five Forces**.
- Maintain evidence-based neutrality; avoid assumptions.
- Present comparative insights in concise visual or tabular formats.
- Conclude with strategic recommendations.

**Expected Output Format and Examples (if applicable)**
1. **Executive Summary** – Overview of competitive landscape.
2. **7S Comparison Matrix** – Our org vs. competitors by element.
3. **Competitive Advantages & Disadvantages Analysis.**
4. **Best Practices and Industry Exemplars.**
5. **Strategic Differentiation Opportunities.**
6. **Competitive Response and Monitoring Plan.**

**Any Additional Notes on Scope or Limitations**
- Only use data explicitly provided or publicly known.
- Avoid speculative financial comparisons.
- Tone: analytical, comparative, strategic.
""",

"PROMPT_13_GAPS": """
**Role Definition**
You are a **Strategic Transformation Consultant** identifying **gaps between the current and desired future state** across all 7S dimensions. Your purpose is to guide leadership on where to focus investments and transformation initiatives.

**Contextual Information**
Inputs include:
- Vision for the future state (3–5 years)
- Current state summary by S
- Strategic objectives and KPIs
- Market trends and stakeholder expectations
Use this information to evaluate transformation readiness and priorities.

**Task Description and Goals**
1. Compare current vs. future state per S element.
2. Quantify and qualify gap severity and closure difficulty.
3. Identify interdependencies and sequencing needs.
4. Recommend a transformation roadmap aligned to business strategy.

**Instructional Guidance and Constraints**
- Use frameworks: **Gap Analysis Matrix**, **McKinsey Horizon Model**, **Prioritization by Impact/Feasibility**.
- Rate gap severity (Critical/High/Medium/Low).
- Separate short-term (quick wins) vs. long-term strategic shifts.
- Keep reasoning explicit and structured.

**Expected Output Format and Examples (if applicable)**
1. **Executive Summary** – Transformation context and gap overview.
2. **Current vs. Future State Table by S Element.**
3. **Gap Severity and Complexity Matrix.**
4. **Resource and Investment Requirements Estimate.**
5. **Transformation Roadmap (Phased).**
6. **Quick Wins vs. Strategic Initiatives.**
7. **Success Metrics and Milestones.**

**Any Additional Notes on Scope or Limitations**
- Do not create financial forecasts.
- Avoid suggesting technology specifics unless context given.
- Maintain practical, action-oriented tone.
""",

"PROMPT_14_INTEGRATION": """
**Role Definition**
You are a **Senior Strategy Integration Consultant** tasked with combining 7S insights with SWOT and PESTLE analyses to form an integrated strategic framework.

**Contextual Information**
Inputs:
- 7S assessment summary
- SWOT and/or PESTLE analysis
- Strategic options under consideration
- Risk factors and opportunity areas
Your goal is to integrate internal (7S) and external (SWOT/PESTLE) perspectives for cohesive strategy formulation.

**Task Description and Goals**
1. Map external opportunities and threats to internal strengths and weaknesses.
2. Evaluate feasibility and risk of each strategic option.
3. Identify where internal capability supports or limits external exploitation.
4. Prioritize initiatives with best strategic fit and readiness.

**Instructional Guidance and Constraints**
- Apply **TOWS Matrix**, **PESTLE Cross-Impact Mapping**, and **Strategic Option Evaluation Matrix**.
- Link insights from 7S internal alignment to external market realities.
- Highlight where misalignment or risk exists.
- Maintain crisp, board-level tone.

**Expected Output Format and Examples (if applicable)**
1. **Executive Summary** – Integrated internal–external perspective.
2. **Capability–Opportunity Alignment Map.**
3. **Strategic Option Evaluation Matrix.**
4. **Risk–Readiness Assessment.**
5. **Prioritized Strategic Initiatives.**
6. **Implementation Feasibility and Governance Recommendations.**
7. **Integrated Dashboard Design (optional).**

**Any Additional Notes on Scope or Limitations**
- Avoid excessive repetition of SWOT/PESTLE details—synthesize instead.
- Focus on actionable integration.
- Keep tone confident, succinct, and senior-level.
""",

"PROMPT_15_EXEC_SUMMARY": """
**Role Definition**
You are a **Senior Partner Consultant** preparing an **executive summary** of a full 7S assessment for the C-suite. Your goal is to synthesize findings into concise, actionable insights suitable for decision-making.

**Contextual Information**
Inputs:
- Full 7S analysis results
- Strategic context and urgency
- Budget and resource constraints
- Board or executive priorities
- Key stakeholder expectations and success criteria
You must condense the full report into a 2–3-page executive deliverable.

**Task Description and Goals**
1. Summarize critical findings and insights from all 7 elements.
2. Identify top strategic imperatives and investment areas.
3. Distinguish between short-term wins and long-term transformations.
4. Present ROI rationale, risk profile, and next steps clearly.

**Instructional Guidance and Constraints**
- Use **Pyramid Principle** (conclusion first, supporting logic after).
- Quantify impact where data allows.
- Prioritize clarity and brevity; every sentence must add value.
- Tone: professional, confident, and insight-driven.

**Expected Output Format and Examples (if applicable)**
1. **Executive Summary (2–3 pages)** – Key insights and context.
2. **Visual Alignment Dashboard** – Optional table or chart showing 7S scores.
3. **Top 10 Recommendations Ranked by Impact and Feasibility.**
4. **Investment and Resource Requirements Summary.**
5. **90-Day, 6-Month, and 1-Year Action Plans.**
6. **Success Metrics and Governance Model.**
7. **Key Risks and Mitigation Strategies.**

**Any Additional Notes on Scope or Limitations**
- Avoid granular details already in technical appendices.
- Write for time-pressed senior executives.
- Maintain polished, high-credibility tone.
""",

"MEGA_PROMPT": """
**Role Definition**
You are a **World-Renowned Management Consultant** (WRC) delivering a **comprehensive 7S organizational assessment**. Act as lead author of a multi-section consulting report designed for executive decision-making and transformation planning.

**Contextual Information**
Use all prior contextual information provided, including:
- Company profile, size, revenue, and market position
- Strategic challenges and recent initiatives
- Internal assessments per S element (Strategy, Structure, Systems, Shared Values, Style, Staff, Skills)
Where information is missing, infer logical assumptions based on industry norms.

**Task Description and Goals**
1. Produce a full-length, integrated report analyzing all 7S elements.
2. Identify misalignments and their systemic impacts.
3. Propose targeted interventions with ROI justification.
4. Deliver executive-level narrative suitable for board presentation.

**Instructional Guidance and Constraints**
- Apply professional consulting frameworks: McKinsey 7S, Balanced Scorecard, Kotter’s Change Model.
- Think step-by-step; provide evidence before conclusions.
- Use structured Markdown headings and subheadings.
- Maintain a cohesive story from diagnosis → insight → recommendation.
- Avoid hallucination: infer only with logical consistency.

**Expected Output Format and Examples (if applicable)**
1. **Executive Summary** – 2–3 pages summarizing major findings and recommendations.
2. **Detailed 7S Analysis** – Each S in its own section (2–3 paragraphs minimum each).
3. **Interdependencies Overview** – Narrative of reinforcing/conflicting dynamics.
4. **Strategic Recommendations** – Prioritized by impact and feasibility.
5. **Implementation Plan** – Phased actions with responsible functions.
6. **Governance & Metrics** – Success tracking model.
7. **Appendices** – Tables, charts, and optional data references.

**Any Additional Notes on Scope or Limitations**
- Do not include fictional data or financial figures.
- Maintain neutral, evidence-based judgment.
- Style: formal yet conversational, resembling a McKinsey final report.
"""
})

# Map bracketed placeholders to ORG fields
PLACEHOLDER_MAP = {
    "[NAME]": ORG.get("name",""),
    "[INDUSTRY]": ORG.get("industry",""),
    "[NUMBER]": ORG.get("size_employees",""),
    "[REVENUE]": ORG.get("annual_revenue",""),
    "[LOCATIONS]": ORG.get("locations",""),
    "[BRIEF DESCRIPTION]": ORG.get("current_context",""),
    "[LIST 3-5]": ORG.get("key_challenges",""),
    "[DESCRIBE]": ORG.get("recent_changes",""),
    "[PROVIDE]": ORG.get("strategy_statement",""),
    "[LIST]": ORG.get("strategic_goals",""),
    "[STATEMENT]": ORG.get("value_proposition",""),
    "[DESCRIBE HIERARCHY]": ORG.get("org_chart",""),
    "[NUMBERS]": ORG.get("span_of_control",""),
    "[IF AVAILABLE]": "",
    "[AVERAGES]": "",
    "[BREAKDOWN]": "",
    "[DATA]": "",
    "[POSITION VS MARKET]": "",
    "[ASSESS 1-5]": "",
    "[CURRENT STATE]": "",
    "[ASSESS]": "",
    "[SUMMARIZE]": "",
    "[ANALYZE]": "",
    "[IDENTIFY]": "",
    "[SPECIFY]": ""
}

def materialize_prompt(key: str) -> str:
    text = PROMPTS[key]
    for k,v in PLACEHOLDER_MAP.items():
        text = text.replace(k, str(v if v else f"N/A: {k}"))
    return text


In [9]:
# ===============================================================
# 🧠 OLLAMA CHAT HELPERS — Auto-detects generate/chat endpoint
# ===============================================================
import requests, json, time
from typing import Dict

TEMPERATURE, TOP_P, REPEAT_PENALTY, TIMEOUT_S = 0.4, 0.9, 1.1, 860

# ---------------------------------------------------------------
# 1️⃣ Detect endpoint availability and set USE_GENERATE
# ---------------------------------------------------------------
BASE = "http://127.0.0.1:11434"
GEN  = f"{BASE}/api/generate"
CHAT = f"{BASE}/api/chat"

def detect_generate_support():
    try:
        r = requests.options(GEN, timeout=3)
        if r.status_code in (200, 204):
            return True
    except Exception:
        pass
    return False

USE_GENERATE = detect_generate_support()
print(f"⚙️ Using {'/api/generate' if USE_GENERATE else '/api/chat'} endpoint for inference.")

# ---------------------------------------------------------------
# 2️⃣ Ask LLM — general wrapper
# ---------------------------------------------------------------
def ask_llm(model: str, sys: str, user: str) -> str:
    """Send message to Ollama and return text output."""
    try:
        if USE_GENERATE:
            # Simpler, faster API (no chat format)
            prompt = f"<<SYS>>\n{sys}\n<</SYS>>\n\n{user}"
            r = requests.post(
                GEN,
                json={
                    "model": model,
                    "prompt": prompt,
                    "stream": False,
                    "options": {
                        "temperature": TEMPERATURE,
                        "top_p": TOP_P,
                        "repeat_penalty": REPEAT_PENALTY
                    },
                },
                timeout=TIMEOUT_S,
            )
            r.raise_for_status()
            return (r.json().get("response") or "").strip()
        else:
            # Chat-style API
            r = requests.post(
                CHAT,
                json={
                    "model": model,
                    "messages": [
                        {"role": "system", "content": sys},
                        {"role": "user", "content": user},
                    ],
                    "stream": False,
                    "options": {
                        "temperature": TEMPERATURE,
                        "top_p": TOP_P,
                        "repeat_penalty": REPEAT_PENALTY,
                    },
                },
                timeout=TIMEOUT_S,
            )
            r.raise_for_status()
            return ((r.json().get("message") or {}).get("content") or "").strip()

    except requests.exceptions.Timeout:
        return "⚠️ Request timed out — model took too long."
    except Exception as e:
        return f"❌ Error querying model {model}: {e}"

# ---------------------------------------------------------------
# 3️⃣ Multi-model consensus synthesis
# ---------------------------------------------------------------
def synthesize_consensus(primary_model: str, prompt_name: str, per_model: Dict[str, str]) -> str:
    """Combine responses from multiple models into one cohesive expert synthesis."""
    sys = (
        "You are a senior WRC consultant. "
        "Synthesize multiple expert drafts into one coherent, non-redundant, executive-ready report."
    )
    user = f"""Prompt: {prompt_name}

Combine the following {len(per_model)} model answers into one structured response.
Do NOT mention that this is a synthesis. Present it like a polished consultant report.

Guidelines:
- Keep the best points from each.
- Merge ideas logically.
- If answers conflict, show trade-offs and state your recommendation.
- Use concise sections and bulleted clarity.

=== MODEL ANSWERS ===
{json.dumps(per_model, indent=2, ensure_ascii=False)}
"""
    return ask_llm(primary_model, sys, user)


⚙️ Using /api/chat endpoint for inference.


In [14]:
# ===============================================================
# 🧠 RUN SELECTED PROMPTS ACROSS MULTIPLE MODELS (with timing)
# ===============================================================
import time
from typing import Dict, List

def run_prompt_across_models(prompt_key: str, models: List[str]) -> Dict[str, dict]:
    """
    Runs a given prompt across multiple models and tracks generation times.
    Returns: { model_name: { "response": str, "duration_s": float, "chars": int } }
    """
    sys_prompt = "You are precise and to-the-point."
    user_prompt = materialize_prompt(prompt_key)
    results = {}

    print(f"\n🚀 Running prompt: {prompt_key}")
    for model in models:
        try:
            start = time.time()
            response = ask_llm(model, sys_prompt, user_prompt)
            duration = round(time.time() - start, 2)
            results[model] = {
                "response": response,
                "duration_s": duration,
                "chars": len(response)
            }
            print(f"  ✓ {model:<12} | {len(response):>5} chars | {duration:>6.2f}s")
        except Exception as e:
            results[model] = {"response": f"(error: {e})", "duration_s": 0, "chars": 0}
            print(f"  ✗ {model:<12} | error: {e}")

    return results


# ---------------------------
# MAIN EXECUTION
# ---------------------------
results = {}
consensus = {}

print("\n🧠 Running prompts:", ", ".join(RUN_PROMPTS))
total_start = time.time()

for key in RUN_PROMPTS:
    model_outputs = run_prompt_across_models(key, REQUESTED_MODELS)
    results[key] = model_outputs
    consensus[key] = synthesize_consensus(PRIMARY_MODEL, key, model_outputs)

total_duration = round(time.time() - total_start, 2)
print(f"\n⏱️ All prompts completed in {total_duration}s")

# ---------------------------
# 🧾 SUMMARY REPORT
# ---------------------------
print("\n📊 Performance Summary (per model):")
summary = {}
for k, model_data in results.items():
    for m, d in model_data.items():
        if m not in summary: summary[m] = []
        summary[m].append(d["duration_s"])

for m, times in summary.items():
    avg_t = sum(times)/len(times)
    print(f"  {m:<12}: avg {avg_t:>6.2f}s across {len(times)} runs")



🧠 Running prompts: PROMPT_1_FULL_7S, PROMPT_2_STRATEGY, PROMPT_3_STRUCTURE, PROMPT_4_SYSTEMS, PROMPT_5_SHARED_VALUES, PROMPT_6_SKILLS, PROMPT_7_STYLE, PROMPT_8_STAFF, PROMPT_9_ALIGNMENT, PROMPT_10_CHANGE, PROMPT_11_DIGITAL, PROMPT_12_BENCH, PROMPT_13_GAPS, PROMPT_14_INTEGRATION, PROMPT_15_EXEC_SUMMARY, MEGA_PROMPT
  ✓ PROMPT_1_FULL_7S via gpt-oss: 287 chars
  ✓ PROMPT_1_FULL_7S via mistral: 287 chars
  ✓ PROMPT_1_FULL_7S via llama3: 286 chars
  ✓ PROMPT_1_FULL_7S via gemma2: 286 chars
  ✓ PROMPT_1_FULL_7S via qwen3:14b: 289 chars
  ✓ PROMPT_2_STRATEGY via gpt-oss: 287 chars
  ✓ PROMPT_2_STRATEGY via mistral: 287 chars
  ✓ PROMPT_2_STRATEGY via llama3: 286 chars
  ✓ PROMPT_2_STRATEGY via gemma2: 286 chars
  ✓ PROMPT_2_STRATEGY via qwen3:14b: 289 chars
  ✓ PROMPT_3_STRUCTURE via gpt-oss: 287 chars
  ✓ PROMPT_3_STRUCTURE via mistral: 287 chars
  ✓ PROMPT_3_STRUCTURE via llama3: 286 chars
  ✓ PROMPT_3_STRUCTURE via gemma2: 286 chars
  ✓ PROMPT_3_STRUCTURE via qwen3:14b: 289 chars
  ✓ PROM

In [ ]:
# ---------------------------
# 6) SAVE OUTPUTS
# ---------------------------
bundle = {
    "metadata": {
        "generated_at_utc": datetime.datetime.now(datetime.timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ"),
        "ollama_host": BASE,
        "models": REQUESTED_MODELS,
        "primary_model": PRIMARY_MODEL
    },
    "org_inputs": ORG,
    "raw_per_prompt": results,
    "consensus_per_prompt": consensus
}

with open(BUNDLE_JSON, "w", encoding="utf-8") as f:
    json.dump(bundle, f, indent=2, ensure_ascii=False)

with open(REPORT_MD, "w", encoding="utf-8") as f:
    f.write(f"# 7S Report — Consensus (models: {', '.join(REQUESTED_MODELS)})\n\n")
    f.write("## Organization\n")
    f.write("```json\n" + json.dumps(ORG, indent=2, ensure_ascii=False) + "\n```\n\n")
    for key in RUN_PROMPTS:
        f.write(f"## {key}\n\n")
        f.write(consensus.get(key,"").strip() + "\n\n")
        f.write("<details><summary>Model answers</summary>\n\n")
        f.write("```json\n" + json.dumps(results.get(key,{}), indent=2, ensure_ascii=False) + "\n```\n")
        f.write("</details>\n\n")

print("\n✅ Done.")
print("JSON:", BUNDLE_JSON)
print("MD  :", REPORT_MD)
# ==========================================================================================

In [ ]:
# ======================= DISPLAY 7S REPORT IN-CELL =======================
import os, json, textwrap
from IPython.display import display, Markdown, HTML

OUT_DIR = "/content"
BUNDLE_JSON = os.path.join(OUT_DIR, "7S_bundle.json")
REPORT_MD   = os.path.join(OUT_DIR, "7S_report.md")

def _h2(s):
    return f"\n## {s}\n"
def _h3(s):
    return f"\n### {s}\n"
def _codeblock(label, obj):
    if isinstance(obj, str):
        body = obj
    else:
        body = json.dumps(obj, indent=2, ensure_ascii=False)
    return f"\n**{label}**\n\n```json\n{body}\n```\n"

# 1) If the Markdown report exists, render that directly (richest output)
if os.path.exists(REPORT_MD):
    with open(REPORT_MD, "r", encoding="utf-8") as f:
        md = f.read()
    display(Markdown(md))
else:
    # 2) Otherwise, pretty-print from the JSON bundle
    if not os.path.exists(BUNDLE_JSON):
        raise FileNotFoundError("No report found. Run the main cell first to create /content/7S_bundle.json or /content/7S_report.md.")

    with open(BUNDLE_JSON, "r", encoding="utf-8") as f:
        bundle = json.load(f)

    meta     = bundle.get("metadata", {})
    org      = bundle.get("org_inputs", {})
    consensus = bundle.get("consensus_per_prompt", {})
    raw       = bundle.get("raw_per_prompt", {})

    # Build a friendly Markdown view
    parts = []
    title = f"# 7S Report — Consensus (models: {', '.join(meta.get('models', []))})"
    parts.append(title)
    parts.append(_h2("Organization"))
    parts.append("```json\n" + json.dumps(org, indent=2, ensure_ascii=False) + "\n```")

    # Show consensus outputs section-by-section
    for k in consensus.keys():
        parts.append(_h2(k))
        txt = consensus.get(k, "").strip()
        if not txt:
            parts.append("_No consensus text produced for this prompt._")
        else:
            # Light formatting: ensure lines not too long for the notebook width
            wrapped = "\n".join(textwrap.fill(line, width=100) for line in txt.splitlines())
            parts.append(wrapped)

        # Collapsible raw model answers
        parts.append("\n<details><summary>Model answers</summary>\n\n")
        parts.append("```json\n" + json.dumps(raw.get(k, {}), indent=2, ensure_ascii=False) + "\n```")
        parts.append("\n</details>\n")

    display(Markdown("\n".join(parts)))
# ========================================================================
